In [5]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

In [6]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [7]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [10]:
analize = Analizer(0.9)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,mape,rmse,r2_adj,rsd,aic,bic,Architecture
6,model_22_9_0,0.999840,0.825152,0.998620,0.999730,0.999428,0.000227,0.248171,0.001072,0.000346,0.000709,0.013294,0.015063,1.000007,0.015704,1222.782182,1957.764305,"Hidden Size=[30, 16], regularizer=0.2, learnin..."
7,model_22_9_1,0.999840,0.825153,0.998620,0.999730,0.999428,0.000227,0.248171,0.001072,0.000346,0.000709,0.013295,0.015063,1.000007,0.015704,1222.782127,1957.764249,"Hidden Size=[30, 16], regularizer=0.2, learnin..."
8,model_22_9_2,0.999840,0.825153,0.998620,0.999730,0.999428,0.000227,0.248171,0.001072,0.000346,0.000709,0.013293,0.015063,1.000007,0.015704,1222.782076,1957.764198,"Hidden Size=[30, 16], regularizer=0.2, learnin..."
9,model_22_9_3,0.999840,0.825153,0.998620,0.999730,0.999428,0.000227,0.248171,0.001072,0.000346,0.000709,0.013294,0.015063,1.000007,0.015704,1222.782058,1957.764181,"Hidden Size=[30, 16], regularizer=0.2, learnin..."
10,model_22_9_4,0.999840,0.825153,0.998620,0.999730,0.999428,0.000227,0.248171,0.001072,0.000346,0.000709,0.013294,0.015063,1.000007,0.015704,1222.782015,1957.764137,"Hidden Size=[30, 16], regularizer=0.2, learnin..."
11,model_22_9_5,0.999840,0.825153,0.998620,0.999730,0.999428,0.000227,0.248171,0.001072,0.000346,0.000709,0.013293,0.015063,1.000007,0.015705,1222.781987,1957.764109,"Hidden Size=[30, 16], regularizer=0.2, learnin..."
12,model_22_9_6,0.999840,0.825153,0.998620,0.999730,0.999428,0.000227,0.248170,0.001072,0.000346,0.000709,0.013292,0.015063,1.000007,0.015705,1222.781976,1957.764099,"Hidden Size=[30, 16], regularizer=0.2, learnin..."
13,model_22_9_7,0.999840,0.825153,0.998620,0.999730,0.999428,0.000227,0.248170,0.001072,0.000346,0.000709,0.013292,0.015063,1.000007,0.015705,1222.781943,1957.764065,"Hidden Size=[30, 16], regularizer=0.2, learnin..."
14,model_22_9_10,0.999840,0.825153,0.998620,0.999730,0.999428,0.000227,0.248170,0.001072,0.000346,0.000709,0.013292,0.015064,1.000007,0.015705,1222.781915,1957.764037,"Hidden Size=[30, 16], regularizer=0.2, learnin..."
15,model_22_9_9,0.999840,0.825153,0.998620,0.999730,0.999428,0.000227,0.248170,0.001072,0.000346,0.000709,0.013291,0.015064,1.000007,0.015705,1222.781910,1957.764032,"Hidden Size=[30, 16], regularizer=0.2, learnin..."
